# Abstract

The 80/20 rule has proven true for many businesses–only a small percentage of customers produce most of the revenue. Due to this, marketing teams are challenged to make appropriate investments in promotional strategies.

In this project, I analyzed a Google Merchandise Store (also known as GStore, where Google swag is sold) customer dataset to predict based on the web behavior of consumers on the Google Merchadise Online store, their likelihood or propensity to convert. 

Propensity models make true predictions about a customer’s future behavior. With propensity models you can truly anticipate a customers’ future behavior.

Here we focus on building a combination of a Propensity to convert and a Propensity to buy model that can influence the kind of marketing campaigns we adopt and who we decide to target (predicted converters vs non-converters) leading to spend optimizations that eventually increase the ROI on digital marketing campaigns. 

The goal of building such propensity models is to attain more actionable operational changes and a better use of marketing budgets for those companies who choose to use data analysis on top of GA data.

# Introduction

Propensity models are what most people think of when they hear “predictive analytics” in the Marketing world. Propensity models make true predictions about a customer’s future behavior and help you truly anticipate a customers’ future behavior.

There are a number of types of Propensity models:

**Predicted lifetime value**

Algorithms can predict how much a customer will spend with you long before customers themselves realizes this. At the moment a customer makes their first purchase you may know a lot more than just their initial transaction record: you may have email and web engagement data for example, as well as demographic and geographic information. By comparing a customer to many others who came before him (or her) you can predict with a high degree of accuracy their future lifetime value. This information is extremely valuable as it allows you to make value based marketing decisions. For example, it makes sense to invest more in those acquisition channels and campaigns that produce customers with the highest predicted lifetime value.

**Predicted share of wallet**

With predicted share of wallet models you can estimate what percentage of a person’s category spend you currently have achieved. For example if a customer spends $100 with you on groceries, is this 10% or 90% of their grocery spending for a given year? Knowing this allows you to see where future revenue potential is within your existing customer base and to design campaigns to capture this revenue. 

**Propensity to convert**

The propensity to convert model can predict the likelihood for a customer to accept your offer. This model can be used for direct mail campaigns where the cost of marketing is high for example. In this case you only want to send the offers to customers with a high propensity to convert.

**Propensity to buy**

The propensity to buy model tells you which customers are ready to make their purchase: so you can find who to target. Moreover, once you know who is ready and who is not helps you provide the right aggression in your offer. Those that are likely to buy won't need high discounts while customers who are not likely to buy may need a more aggressive offer, thereby bringing you incremental revenue.

**Propensity to engage**

A propensity to engage model predicts how likely it is that a customer will click on your email links. Armed with this information you can decide not to send an email to a certain “low likelihood to click” segment.

**Propensity to unsubscribe**

A propensity to unsubscribe model predicts how likely it is that a customer will unsubscribe from your email list at any given point in time. Armed with this information you can optimize email frequency. For “high likelihood to unsubscribe” segments you should decrease send frequency, whereas for “low likelihood to unsubscribe” segments you can increase email send frequency.

**Propensity to churn**

The propensity to churn model tells you which active customers are at risk, so you know which high value, at risk customers to put on your watch list and reach out.

Often propensity models can be combined to make campaign decisions. For example, you may want to do an aggressive customer win back campaign for customers who have both a high likelihood to churn and a high predicted lifetime value.

Here we focus on building a combination of a Propensity to convert and a Propensity to buy model that can influence the kind of marketing campaigns we adopt and who we decide to target (predicted converters vs non-converters) leading to spend optimizations that eventually increase the ROI on digital marketing campaigns

# Data and Wrangling

Data used to build the Propensity to Convert model was obtained via the following Kaggle competition: https://www.kaggle.com/c/ga-customer-revenue-prediction

The train_v2.csv file contains the columns listed under **Data Fields**. Each row in the dataset is one visit to the Google Merchadise Online store and contains user transactions from August 1st 2016 to April 30th 2018

The dataset contains multiple columns which contain JSON blobs of varying depth. In one of those JSON columns, 'totals', the sub-column transactionRevenue contains the revenue information we use to build our targeting variable used for training and prediction:
* 1.0- Signifies a converter who generated revenue
* 0.0- Signifies a non-converter who did not generate revenue

## Data Fields

* **fullVisitorId -** A unique identifier for each user of the Google Merchandise Store.
* **channelGrouping -** The channel via which the user came to the Store.
* **date -** The date on which the user visited the Store.
* **device -** The specifications for the device used to access the Store.
* **geoNetwork -** This section contains information about the geography of the user.
* **socialEngagementType -** Engagement type, either "Socially Engaged" or "Not Socially Engaged".
* **totals -** This section contains aggregate values across the session.
* **trafficSource -** This section contains information about the Traffic Source from which the session originated.
* **visitId -** An identifier for this session. This is part of the value usually stored as the _utmb cookie. This is only unique to the user. For a completely unique ID, you should use a combination of fullVisitorId and visitId.
* **visitNumber -** The session number for this user. If this is the first session, then this is set to 1.
* **visitStartTime -** The timestamp (expressed as POSIX time).
* **hits -** This row and nested fields are populated for any and all types of hits. Provides a record of all page visits.
* **customDimensions -** This section contains any user-level or session-level custom dimensions that are set for a session. This is a repeated field and has an entry for each dimension that is set.
* **totals -** This set of columns mostly includes high-level aggregate data.

# Workflow

1. [Pre-processing Pipeline to be Applied to the Data](#Preprocesing)
    * Hits and customDimensions do not have data that is relevant to predicting the target variable. Droping these columns:
    * normalizing the columns in json format and appending columns to original dataframe
    * Creating training label from the 'totals.transactionRevenue' column
    * Separating the converter and non-converter data to adress the class imbalance later in the processing pipeline
1. [Iterating Over the Data in Chunks Using nrows and skiprows](#BatchProcessing)
1. [Addressing Class Imbalance](#ClassImb)
    * Reading in all the data.
    * Set seed for random sampling.
    * Downsampling (without replacement) the non-converter data to arrive at a sample size equal to that of the converter data.
    * Split dataset into training and universal test data
    * Keep universal test data aside and ensure this data is noe present in future samples
    * Train model and save the model
    * Repeat steps 1-5 with different seeds to create many such training data draws.
    * Select model with heightest accuracy/ use weighted average of each model (based on accuracy) for prediction. Use same universal test dataset to determine accuracy of each model
1. [Converter Data (Label 1)](#GenLab)
1. [Split into Training and Test Data (80/20)](#Split)
1. [One-Hot Encoding of Features (Categorical Variables)](#OHE)
1. [Recursive Feature Elimination: Feature Importance](#RFE)
1. [Training the Model](#Training)
    * [Logistic Regression Model](#logreg)
    * [Random Forest Model](#RF)
1. [Exploratory Data Analysis to Create Scalable Data Pre-processing Pipeline and Feature Selection](#EDA)
    * Reading a subset of the data (.csv file train_v2.csv), about 20000 rows into memory
    * Converting fullVisitorId', 'visitId', 'visitStartTime' columns into string datatypes and flattening the 'totals', 'device', 'geoNetwork', 'trafficSource' columns which are in json format
    * Inspecting the contents of the columns containing json objects
    * Feature Selection
    * Dropping irrelevant features (low variance) and features with a large volume of missing data via inspection
    * Inspecting the contents of columns to determine which columns are worth keeping. Only columns with high variance is retained. Columns with missing data or low variance data are omitted.
    * Creating training label from the 'totals.transactionRevenue' column
    * We notice that only a small fraction of the visitors convert. This leads to a class imbalance that needs to be addressed before training our propensity to convert model.

# Environment

Jupyter Notebook which is an open-source web application that allows you to create and share documents that contain live code, equations, visualizations and explanatory text is used here for data cleaning and transformation, statistical modeling and machine learning.

# Challenges

* Dealing with a large dataset: Cloud vs local execution

* Mixed dataset with regular text columns as well as json blobs

* Dealing with the 80/20 rule that leads to class imbalance in training data

# Conclusion

Two Propensity to convert models were built using relativey low complexity machine learning algorithms, namely Logistic Regression and Random Forest. They had high accuracy scores of 95.56% and 95.73% on the test data.

The Random Forest model performed slightly better than the Logistic Regression model and is preferred due to faster training time as a result of paralization of the training process.

# Future work

Building out a regression model that predicts expected revenue for converters so as to better assign marketing budgets.

# References

https://blog.agilone.com/the-definitive-guide-to-predictive-analytics-models-for-marketing

# Code

In [1]:
import pandas as pd
import json
import pandas.io.json as pdjson
import matplotlib.pyplot as plt
import random
import math
import numpy as np
from numpy import array
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [2]:
path='C:/Users/niharika/Desktop/Vainu/train_v2.csv'

In [3]:
n = sum(1 for line in open(path, encoding='iso-8859-1')) - 1 # excluding header from count of number of rows 'n'

In [4]:
print('training.csv has {} rows'.format(n))

training.csv has 1708337 rows


# Pre-processing Pipeline to be Applied to the Data <a name="Preprocesing"></a>

In [5]:
def preprocesssing(chunk, iteration):
    
    print('processing iteration {}'.format(iteration))
    
    data_train= chunk
    
    # Hits and customDimensions do not have data that is relevant to predicting the target variable. Droping these columns:
    data_train = data_train.drop(['hits', 'customDimensions'], axis=1)


    # normalizing the columns in json format and appending columns to original dataframe
    columns=['totals', 'device', 'geoNetwork', 'trafficSource']
    for column in columns:
        column_as_df = pdjson.json_normalize(data_train[column])
        column_as_df.columns = ["{}.{}".format(column, subcolumn) for subcolumn in column_as_df.columns]
        data_train = data_train.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

    if 'totals.transactionRevenue' not in data_train.columns:
        print('label not found')
        
        return
    
    else:
        print('label found')
        # Feature Engineering and Feature Selection
        ## Adjusting Datatypes and filling NaN and converting 'totals' data to float
        data_train[['totals.bounces', 'totals.hits',
           'totals.newVisits', 'totals.pageviews', 'totals.sessionQualityDim',
           'totals.timeOnSite', 'totals.totalTransactionRevenue',
           'totals.transactionRevenue', 'totals.transactions', 'totals.visits']] = data_train[['totals.bounces', 'totals.hits',
           'totals.newVisits', 'totals.pageviews', 'totals.sessionQualityDim',
           'totals.timeOnSite', 'totals.totalTransactionRevenue',
           'totals.transactionRevenue', 'totals.transactions', 'totals.visits']].fillna(0).astype('float') 

        ## Dropping irrelevant features (low variance) and features with a large volume of missing data via inspection 
        data_train = data_train.drop([
            'socialEngagementType', 
          'device.browserSize', 
          'device.browserVersion', 
          'device.flashVersion', 
          'device.language',
          'device.mobileDeviceInfo',
          'device.mobileDeviceBranding',
          'device.isMobile',
          'device.mobileDeviceInfo',                      
           'device.mobileDeviceMarketingName', 
            'device.mobileDeviceModel',
           'device.mobileInputSelector', 
            'device.operatingSystemVersion', 
            'device.screenColors',
           'device.screenResolution', 
            'geoNetwork.city',
            'geoNetwork.cityId',
            'geoNetwork.latitude',
            'geoNetwork.longitude', 
            'geoNetwork.metro',
            'geoNetwork.networkDomain',
            'geoNetwork.networkLocation',
            'geoNetwork.region',
            'trafficSource.adContent',
            'trafficSource.adwordsClickInfo.adNetworkType',
            'trafficSource.adwordsClickInfo.criteriaParameters',
            'trafficSource.adwordsClickInfo.gclId',
            'trafficSource.adwordsClickInfo.isVideoAd',
            'trafficSource.adwordsClickInfo.page',
            'trafficSource.adwordsClickInfo.slot',
            'trafficSource.campaign',
            'trafficSource.isTrueDirect',
            'trafficSource.keyword',
            'trafficSource.medium',
            'trafficSource.referralPath',
            'trafficSource.source'], axis=1) 

        ## grouping data 
        data_train_grouped= data_train.groupby(['date','channelGrouping', 'fullVisitorId', 'visitId', 'device.browser', 'device.deviceCategory',
            'device.operatingSystem','geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.subContinent']).sum()

        data_train_grouped= data_train_grouped.reset_index()

        # Creating training label from the 'totals.transactionRevenue' column
        data_train_grouped['label']=data_train_grouped['totals.transactionRevenue']
        for i in data_train_grouped[data_train_grouped['label']>0].index :
            data_train_grouped.loc[i, 'label']= 1.0



        '''We notice that only a small fraction of the visitors convert. 
        This leads to a class imbalance that needs to be addressed before training our propensity to convert model.'''

        # Separating the converter and non-converter data to adress the class imbalance later in the processing pipeline
        data_converter= data_train_grouped[data_train_grouped['label']>0]
        data_nonconverter= data_train_grouped[data_train_grouped['label']<=0]

        data_converter.to_csv('data_converter'+'_'+str(iteration)+'.csv', index=False)
        data_nonconverter.to_csv('data_nonconverter'+'_'+str(iteration)+'.csv', index=False)

        print('iteration {}/86: files processed, separated into converter and non-converter and output'.format(iteration))

        return

# Iterating Over the Data in Chunks Using nrows and skiprows <a name="BatchProcessing"></a>

In [28]:
for count in range(0,math.ceil(n/20000)):
    chunk= pd.read_csv(path, nrows=20000, skiprows=range(1, 20000*count),header=0, parse_dates=[2], dtype={'fullVisitorId': object,'visitId': object, 'visitStartTime':object}, converters={'totals':json.loads, 'device':json.loads, 'geoNetwork':json.loads, 'trafficSource':json.loads})
    preprocesssing(chunk, count)

processing iteration 0
label found
iteration 0/86: files processed, separated into converter and non-converter and output
processing iteration 1
label found
iteration 1/86: files processed, separated into converter and non-converter and output
processing iteration 2
label found
iteration 2/86: files processed, separated into converter and non-converter and output
processing iteration 3
label found
iteration 3/86: files processed, separated into converter and non-converter and output
processing iteration 4
label found
iteration 4/86: files processed, separated into converter and non-converter and output
processing iteration 5
label found
iteration 5/86: files processed, separated into converter and non-converter and output
processing iteration 6
label found
iteration 6/86: files processed, separated into converter and non-converter and output
processing iteration 7
label found
iteration 7/86: files processed, separated into converter and non-converter and output
processing iteration 8
l

iteration 66/86: files processed, separated into converter and non-converter and output
processing iteration 67
label found
iteration 67/86: files processed, separated into converter and non-converter and output
processing iteration 68
label found
iteration 68/86: files processed, separated into converter and non-converter and output
processing iteration 69
label found
iteration 69/86: files processed, separated into converter and non-converter and output
processing iteration 70
label found
iteration 70/86: files processed, separated into converter and non-converter and output
processing iteration 71
label found
iteration 71/86: files processed, separated into converter and non-converter and output
processing iteration 72
label found
iteration 72/86: files processed, separated into converter and non-converter and output
processing iteration 73
label found
iteration 73/86: files processed, separated into converter and non-converter and output
processing iteration 74
label found
iteratio

# Addressing Class Imbalance <a name="ClassImb"></a>

1. Reading in all the data.
1. Set seed for random sampling.
1. Downsampling (without replacement) the non-converter data to arrive at a sample size equal to that of the converter data.
1. Split dataset into training and universal test data
1. Keep universal test data aside and ensure this data is noe present in future samples
1. Train model and save the model
1. Repeat steps 1-5 with different seeds to create many such training data draws.
1. Select model with heightest accuracy/ use weighted average of each model (based on accuracy) for prediction. Use same universal test dataset to determine accuracy of each model



## Converter Data (Label 1) <a name="GenLab"></a>

In [6]:
path='C:/Users/niharika/Desktop/Vainu/data_converter_0.csv'
df_data_converter= pd.read_csv(path, header=0, parse_dates=[2], dtype={'fullVisitorId': object,'visitId': object, 'visitStartTime':object})


for count in range(1,math.ceil(n/20000)):
    path='C:/Users/niharika/Desktop/Vainu/data_converter_'+str(count)+'.csv'
    df_data= pd.read_csv(path, header=0, parse_dates=[2], dtype={'fullVisitorId': object,'visitId': object, 'visitStartTime':object})    
    df_data_converter= df_data_converter.append(df_data, ignore_index=True, verify_integrity=True)
    


In [7]:
df_data_converter.shape   

(18514, 22)

## Non-Converter Data (Label 0)

In [8]:
path='C:/Users/niharika/Desktop/Vainu/data_nonconverter_0.csv'
df_data_nonconverter= pd.read_csv(path, header=0, parse_dates=[2], dtype={'fullVisitorId': object,'visitId': object, 'visitStartTime':object})


for count in range(1,math.ceil(n/20000)):
    path='C:/Users/niharika/Desktop/Vainu/data_nonconverter_'+str(count)+'.csv'
    df_data= pd.read_csv(path, header=0, parse_dates=[2], dtype={'fullVisitorId': object,'visitId': object, 'visitStartTime':object})
    df_data_nonconverter= df_data_nonconverter.append(df_data, ignore_index=True, verify_integrity=True)

In [9]:
df_data_nonconverter.shape  

(1689824, 22)

In [10]:
df_data_nc_downsample= df_data_nonconverter.sample(n=18514, replace=False, random_state=10)

In [11]:
df_data_nc_downsample.shape

(18514, 22)

In [12]:
df_data_combined= df_data_nc_downsample.append(df_data_converter, ignore_index=True, verify_integrity=True)

In [13]:
df_data_combined.shape #balanced dataset

(37028, 22)

# Split into Training and Test Data (80/20) <a name="Split"></a>

In [14]:
df_data_combined.columns

Index(['date', 'channelGrouping', 'fullVisitorId', 'visitId', 'device.browser',
       'device.deviceCategory', 'device.operatingSystem',
       'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.subContinent',
       'visitNumber', 'totals.bounces', 'totals.hits', 'totals.newVisits',
       'totals.pageviews', 'totals.sessionQualityDim', 'totals.timeOnSite',
       'totals.totalTransactionRevenue', 'totals.transactionRevenue',
       'totals.transactions', 'totals.visits', 'label'],
      dtype='object')

In [15]:
df_data_combined=df_data_combined.drop(['totals.totalTransactionRevenue', 'totals.transactionRevenue',
       'totals.transactions'], axis=1)

In [16]:
np.random.seed(seed=12)

In [17]:
msk = np.random.rand(len(df_data_combined)) < 0.8

train = df_data_combined[msk]

test = df_data_combined[~msk]

In [18]:
len(test)

7455

In [19]:
X_test= test[['date', 'channelGrouping', 'fullVisitorId', 'visitId', 'device.browser',
       'device.deviceCategory', 'device.operatingSystem',
       'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.subContinent',
       'visitNumber', 'totals.bounces', 'totals.hits', 'totals.newVisits',
       'totals.pageviews', 'totals.sessionQualityDim', 'totals.timeOnSite', 'totals.visits']]
Y_test= test['label']

In [20]:
len(train)

29573

In [21]:
X_train= train[['date', 'channelGrouping', 'fullVisitorId', 'visitId', 'device.browser',
       'device.deviceCategory', 'device.operatingSystem',
       'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.subContinent',
       'visitNumber', 'totals.bounces', 'totals.hits', 'totals.newVisits',
       'totals.pageviews', 'totals.sessionQualityDim', 'totals.timeOnSite', 'totals.visits']]
Y_train= train['label']

In [22]:
X_test=X_test.set_index(['date', 'fullVisitorId', 'visitId'])
X_train=X_train.set_index(['date', 'fullVisitorId', 'visitId'])

## One-Hot Encoding of Features (Categorical Variables) <a name="OHE"></a>

In [23]:
X_trainDict= X_train.T.to_dict().values()
X_testDict= X_test.T.to_dict().values()

vec = DictVectorizer()

X_train_array= vec.fit_transform(X_trainDict).toarray()
X_test_array= vec.transform(X_testDict).toarray()

# Recursive Feature Elimination: Feature Importance <a name="RFE"></a>

In [24]:
logreg = LogisticRegression(solver='liblinear')
rfe = RFE(logreg, n_features_to_select=15)
rfe = rfe.fit(X_train_array, Y_train.ravel())

In [27]:
print ("Features sorted by their rank:")
print (sorted(zip(map(lambda x: round(x, 2), rfe.ranking_), X_train.columns)))

Features sorted by their rank:
[(1, 'channelGrouping'), (1, 'visitNumber'), (19, 'geoNetwork.country'), (20, 'device.browser'), (62, 'totals.pageviews'), (68, 'geoNetwork.subContinent'), (78, 'device.deviceCategory'), (79, 'geoNetwork.continent'), (80, 'device.operatingSystem'), (106, 'totals.hits'), (146, 'totals.visits'), (175, 'totals.sessionQualityDim'), (178, 'totals.newVisits'), (180, 'totals.bounces'), (201, 'totals.timeOnSite')]


# Training the Model <a name="Training"></a>

## Logistic Regression Model <a name="logreg"></a>

In [28]:
logreg = LogisticRegression()
logreg.fit(X_train_array, Y_train)

C:\Users\niharika\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [29]:
LR_predictions = logreg.predict(X_test_array)

### Accuracy

In [30]:
LR_predictions

array([0., 0., 0., ..., 1., 1., 1.])

In [34]:
target_names = ['Non-Converters', 'Converters']
print (classification_report(Y_test, LR_predictions, target_names=target_names))
print('accuracy_score is: {0:.2%}'.format(accuracy_score(Y_test, LR_predictions)))
print()
print(pd.crosstab(Y_test, LR_predictions, rownames=["Actual"], colnames=["Predicted"]))

                precision    recall  f1-score   support

Non-Converters       0.97      0.95      0.96      3726
    Converters       0.95      0.97      0.96      3729

     micro avg       0.96      0.96      0.96      7455
     macro avg       0.96      0.96      0.96      7455
  weighted avg       0.96      0.96      0.96      7455

accuracy_score is: 95.56%

Predicted   0.0   1.0
Actual               
0.0        3522   204
1.0         127  3602


## Random Forest Model <a name="RF"></a>

In [55]:
clf = RandomForestClassifier(n_estimators=10, max_depth=10,random_state=0, n_jobs=-1)

In [56]:
clf.fit(X_train_array, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [57]:
RF_predictions = clf.predict(X_test_array)

### Accuracy

In [58]:
RF_predictions

array([0., 0., 0., ..., 1., 1., 1.])

In [99]:
print(classification_report(Y_test, RF_predictions, target_names=target_names))
print('accuracy_score is: {0:.2%}'.format(accuracy_score(Y_test, RF_predictions)))
print()
print(pd.crosstab(Y_test, RF_predictions, rownames=["Actual"], colnames=["Predicted"]))

                precision    recall  f1-score   support

Non-Converters       0.97      0.94      0.96      3726
    Converters       0.94      0.98      0.96      3729

     micro avg       0.96      0.96      0.96      7455
     macro avg       0.96      0.96      0.96      7455
  weighted avg       0.96      0.96      0.96      7455

accuracy_score is: 95.73%

Predicted   0.0   1.0
Actual               
0.0        3498   228
1.0          90  3639


# Appendix

### Exploratory Data Analysis to Create Scalable Data Pre-processing Pipeline and Feature Selection <a name="EDA"></a>

* Reading a subset of the data (.csv file train_v2.csv), about 20000 rows into memory
* Converting fullVisitorId', 'visitId', 'visitStartTime' columns into string datatypes and flattening the 'totals', 'device', 'geoNetwork', 'trafficSource' columns which are in json format

In [69]:
path='C:/Users/niharika/Desktop/Vainu/train_v2.csv'

In [70]:
data_train= pd.read_csv(path, nrows=20000, parse_dates=[2], dtype={'fullVisitorId': object,'visitId': object, 'visitStartTime':object}, converters={'totals':json.loads, 'device':json.loads, 'geoNetwork':json.loads, 'trafficSource':json.loads})

In [71]:
data_train.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,hits,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",2017-10-16,"{'browser': 'Firefox', 'browserVersion': 'not ...",3162355547410993243,"{'continent': 'Europe', 'subContinent': 'Weste...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{'visits': '1', 'hits': '1', 'pageviews': '1',...","{'campaign': '(not set)', 'source': 'google', ...",1508198450,1,1508198450
1,Referral,"[{'index': '4', 'value': 'North America'}]",2017-10-16,"{'browser': 'Chrome', 'browserVersion': 'not a...",8934116514970143966,"{'continent': 'Americas', 'subContinent': 'Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,"{'visits': '1', 'hits': '2', 'pageviews': '2',...",{'referralPath': '/a/google.com/transportation...,1508176307,6,1508176307
2,Direct,"[{'index': '4', 'value': 'North America'}]",2017-10-16,"{'browser': 'Chrome', 'browserVersion': 'not a...",7992466427990357681,"{'continent': 'Americas', 'subContinent': 'Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{'visits': '1', 'hits': '2', 'pageviews': '2',...","{'campaign': '(not set)', 'source': '(direct)'...",1508201613,1,1508201613
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",2017-10-16,"{'browser': 'Chrome', 'browserVersion': 'not a...",9075655783635761930,"{'continent': 'Asia', 'subContinent': 'Western...","[{'hitNumber': '1', 'time': '0', 'hour': '9', ...",Not Socially Engaged,"{'visits': '1', 'hits': '2', 'pageviews': '2',...","{'campaign': '(not set)', 'source': 'google', ...",1508169851,1,1508169851
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",2017-10-16,"{'browser': 'Chrome', 'browserVersion': 'not a...",6960673291025684308,"{'continent': 'Americas', 'subContinent': 'Cen...","[{'hitNumber': '1', 'time': '0', 'hour': '14',...",Not Socially Engaged,"{'visits': '1', 'hits': '2', 'pageviews': '2',...","{'campaign': '(not set)', 'source': 'google', ...",1508190552,1,1508190552


In [72]:
data_train.dtypes

channelGrouping                 object
customDimensions                object
date                    datetime64[ns]
device                          object
fullVisitorId                   object
geoNetwork                      object
hits                            object
socialEngagementType            object
totals                          object
trafficSource                   object
visitId                         object
visitNumber                      int64
visitStartTime                  object
dtype: object

In [73]:
data_train.columns

Index(['channelGrouping', 'customDimensions', 'date', 'device',
       'fullVisitorId', 'geoNetwork', 'hits', 'socialEngagementType', 'totals',
       'trafficSource', 'visitId', 'visitNumber', 'visitStartTime'],
      dtype='object')

* Inspecting the contents of the columns containing json objects 

In [74]:
data_train.loc[1,'customDimensions']

"[{'index': '4', 'value': 'North America'}]"

In [75]:
data_train.loc[1,'device']

{'browser': 'Chrome',
 'browserVersion': 'not available in demo dataset',
 'browserSize': 'not available in demo dataset',
 'operatingSystem': 'Chrome OS',
 'operatingSystemVersion': 'not available in demo dataset',
 'isMobile': False,
 'mobileDeviceBranding': 'not available in demo dataset',
 'mobileDeviceModel': 'not available in demo dataset',
 'mobileInputSelector': 'not available in demo dataset',
 'mobileDeviceInfo': 'not available in demo dataset',
 'mobileDeviceMarketingName': 'not available in demo dataset',
 'flashVersion': 'not available in demo dataset',
 'language': 'not available in demo dataset',
 'screenColors': 'not available in demo dataset',
 'screenResolution': 'not available in demo dataset',
 'deviceCategory': 'desktop'}

In [76]:
data_train.loc[1,'geoNetwork']

{'continent': 'Americas',
 'subContinent': 'Northern America',
 'country': 'United States',
 'region': 'California',
 'metro': 'San Francisco-Oakland-San Jose CA',
 'city': 'Cupertino',
 'cityId': 'not available in demo dataset',
 'networkDomain': '(not set)',
 'latitude': 'not available in demo dataset',
 'longitude': 'not available in demo dataset',
 'networkLocation': 'not available in demo dataset'}

In [77]:
data_train.loc[1,'totals']

{'visits': '1',
 'hits': '2',
 'pageviews': '2',
 'timeOnSite': '28',
 'sessionQualityDim': '2'}

In [78]:
data_train.loc[1,'trafficSource']

{'referralPath': '/a/google.com/transportation/mtv-services/bikes/bike2workmay2016',
 'campaign': '(not set)',
 'source': 'sites.google.com',
 'medium': 'referral',
 'adwordsClickInfo': {'criteriaParameters': 'not available in demo dataset'}}

In [79]:
data_train.loc[1,'hits']

'[{\'hitNumber\': \'1\', \'time\': \'0\', \'hour\': \'10\', \'minute\': \'51\', \'isInteraction\': True, \'isEntrance\': True, \'referer\': \'https://sites.google.com/a/google.com/transportation/mtv-services/bikes/bike2workmay2016\', \'page\': {\'pagePath\': \'/home\', \'hostname\': \'shop.googlemerchandisestore.com\', \'pageTitle\': \'Home\', \'searchKeyword\': \'jersey\', \'searchCategory\': \'(not set)\', \'pagePathLevel1\': \'/home\', \'pagePathLevel2\': \'\', \'pagePathLevel3\': \'\', \'pagePathLevel4\': \'\'}, \'appInfo\': {\'screenName\': \'shop.googlemerchandisestore.com/home\', \'landingScreenName\': \'shop.googlemerchandisestore.com/home\', \'exitScreenName\': \'shop.googlemerchandisestore.com/asearch.html\', \'screenDepth\': \'0\'}, \'exceptionInfo\': {\'isFatal\': True}, \'product\': [], \'promotion\': [{\'promoId\': \'Apparel Row 1\', \'promoName\': \'Apparel\', \'promoCreative\': \'home_main_link_apparel.jpg\', \'promoPosition\': \'Row 1\'}, {\'promoId\': \'Backpacks Row 

In [80]:
# Hits and customDimensions do not have data that is relevant to predicting the target variable. Droping these columns:

data_train = data_train.drop(['hits', 'customDimensions'], axis=1)


# normalizing the columns in json format and appending columns to original dataframe

columns=['device', 'geoNetwork', 'totals', 'trafficSource']

for column in columns:
    column_as_df = pdjson.json_normalize(data_train[column])
    column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
    data_train = data_train.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    
    

In [81]:
data_train

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,device.browserVersion,...,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,2017-10-16,3162355547410993243,Not Socially Engaged,1508198450,1,1508198450,Firefox,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,water bottle,organic,NaN,google
1,Referral,2017-10-16,8934116514970143966,Not Socially Engaged,1508176307,6,1508176307,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,NaN,referral,/a/google.com/transportation/mtv-services/bike...,sites.google.com
2,Direct,2017-10-16,7992466427990357681,Not Socially Engaged,1508201613,1,1508201613,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),True,NaN,(none),NaN,(direct)
3,Organic Search,2017-10-16,9075655783635761930,Not Socially Engaged,1508169851,1,1508169851,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
4,Organic Search,2017-10-16,6960673291025684308,Not Socially Engaged,1508190552,1,1508190552,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
5,Referral,2017-10-16,0166277907528479249,Not Socially Engaged,1508196701,1,1508196701,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(none),/offer/2145,(direct)
6,Referral,2017-10-16,8349655975937271469,Not Socially Engaged,1508152478,1,1508152478,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,NaN,referral,/a/google.com/nest-vision/dropcam-field-tester...,sites.google.com
7,Organic Search,2017-10-16,1332629902468998662,Not Socially Engaged,1508206208,1,1508206208,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
8,Organic Search,2017-10-16,632878546807742341,Not Socially Engaged,1508207516,1,1508207516,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,NaN,google
9,Organic Search,2017-10-16,1259490915281096752,Not Socially Engaged,1508165159,2,1508165159,Safari,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,NaN,google


### Feature Selection 

* Adjusting Datatypes by inspection

In [82]:
data_train.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'socialEngagementType',
       'visitId', 'visitNumber', 'visitStartTime', 'device.browser',
       'device.browserSize', 'device.browserVersion', 'device.deviceCategory',
       'device.flashVersion', 'device.isMobile', 'device.language',
       'device.mobileDeviceBranding', 'device.mobileDeviceInfo',
       'device.mobileDeviceMarketingName', 'device.mobileDeviceModel',
       'device.mobileInputSelector', 'device.operatingSystem',
       'device.operatingSystemVersion', 'device.screenColors',
       'device.screenResolution', 'geoNetwork.city', 'geoNetwork.cityId',
       'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.latitude',
       'geoNetwork.longitude', 'geoNetwork.metro', 'geoNetwork.networkDomain',
       'geoNetwork.networkLocation', 'geoNetwork.region',
       'geoNetwork.subContinent', 'totals.bounces', 'totals.hits',
       'totals.newVisits', 'totals.pageviews', 'totals.sessionQualityDim',
       'totals.tim

In [83]:
data_train.dtypes

channelGrouping                                              object
date                                                 datetime64[ns]
fullVisitorId                                                object
socialEngagementType                                         object
visitId                                                      object
visitNumber                                                   int64
visitStartTime                                               object
device.browser                                               object
device.browserSize                                           object
device.browserVersion                                        object
device.deviceCategory                                        object
device.flashVersion                                          object
device.isMobile                                                bool
device.language                                              object
device.mobileDeviceBranding                     

In [84]:
data_train[['totals.bounces', 'totals.hits',
       'totals.newVisits', 'totals.pageviews', 'totals.sessionQualityDim',
       'totals.timeOnSite', 'totals.totalTransactionRevenue',
       'totals.transactionRevenue', 'totals.transactions', 'totals.visits']] = data_train[['totals.bounces', 'totals.hits',
       'totals.newVisits', 'totals.pageviews', 'totals.sessionQualityDim',
       'totals.timeOnSite', 'totals.totalTransactionRevenue',
       'totals.transactionRevenue', 'totals.transactions', 'totals.visits']].fillna(0).astype('float') #filling NaN and converting to float

* Dropping irrelevant features (low variance) and features with a large volume of missing data via inspection

In [85]:
column_list= ['channelGrouping', 'socialEngagementType','device.browser',
       'device.browserSize', 'device.browserVersion', 'device.deviceCategory',
       'device.flashVersion', 'device.isMobile', 'device.language',
       'device.mobileDeviceBranding', 'device.mobileDeviceInfo',
       'device.mobileDeviceMarketingName', 'device.mobileDeviceModel',
       'device.mobileInputSelector', 'device.operatingSystem',
       'device.operatingSystemVersion', 'device.screenColors',
       'device.screenResolution', 'geoNetwork.city', 'geoNetwork.cityId',
       'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.latitude',
       'geoNetwork.longitude', 'geoNetwork.metro', 'geoNetwork.networkDomain',
       'geoNetwork.networkLocation', 'geoNetwork.region',
       'geoNetwork.subContinent', 'totals.bounces', 'totals.hits',
       'totals.newVisits', 'totals.pageviews', 'totals.sessionQualityDim',
       'totals.timeOnSite', 'totals.totalTransactionRevenue',
       'totals.transactionRevenue', 'totals.transactions', 'totals.visits',
       'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.criteriaParameters',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.isVideoAd',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign',
       'trafficSource.isTrueDirect', 'trafficSource.keyword',
       'trafficSource.medium', 'trafficSource.referralPath',
       'trafficSource.source']

* Inspecting the contents of columns to determine which columns are worth keeping. Only columns with high variance is retained. Columns with missing data or low variance data are omitted.

In [86]:
for i in column_list:
    print('{} :  {}'.format(i, set(data_train[i])))

channelGrouping :  {'Referral', 'Social', 'Direct', 'Affiliates', 'Organic Search', 'Display', 'Paid Search'}
socialEngagementType :  {'Not Socially Engaged'}
device.browser :  {'Opera', 'UC Browser', 'Internet Explorer', 'Chrome', 'Edge', 'Amazon Silk', 'BlackBerry', 'Safari (in-app)', 'Samsung Internet', 'Android Webview', 'Maxthon', 'Nokia Browser', 'Puffin', 'Firefox', 'Android Browser', 'SeaMonkey', 'Opera Mini', 'Safari', 'ADM', 'Nintendo Browser', 'Mozilla Compatible Agent', 'MRCHROME', 'Coc Coc', 'Playstation Vita Browser', 'YaBrowser'}
device.browserSize :  {'not available in demo dataset'}
device.browserVersion :  {'not available in demo dataset'}
device.deviceCategory :  {'tablet', 'mobile', 'desktop'}
device.flashVersion :  {'not available in demo dataset'}
device.isMobile :  {False, True}
device.language :  {'not available in demo dataset'}
device.mobileDeviceBranding :  {'not available in demo dataset'}
device.mobileDeviceInfo :  {'not available in demo dataset'}
device.m

trafficSource.isTrueDirect :  {nan, True}
trafficSource.keyword :  {nan, 'shop shirts', 'googlemerch', 'google store merchandise', '1hZbAqLCbjwfgOH7', 'google + online', '(User vertical targeting)', 'www.google.com bag', 'google online merchandise', 'google t-shirt', 'www you tope', 'google shirts buy', 'youtube shop', '(automatic matching)', 'android clothes', 'www google', 'google youtube', '71817', 'youtube player t shirt', 'yuo utube', 'you tybe', 'google shopping tshirts', 'google Merchandising kosten', 'www.google .com', 'merchdise', 'googel store', 'google developer t shirt india', 'google t-shirt free', 'yoputube', '+google +store +merchandise', 'youtube', 'google t shirt', '1X4Me6ZKNV0zg-jV', 'youtube google', 'Google Merchandise', 'Clear pics of different types of blue t-shirts', 'youtube customize tshirt online', 'google water bottle', 'google sourcing earphones', 'google canada', 'youtube store', 'coca cola lava lamp', 'the office youtube uk', 'youttub', 'tou tube', 'yputub

1. **'channelGrouping': high variance**
1. 'socialEngagementType': low variance
1. **'device.browser': high variance**
1. 'device.browserSize': missing data
1. 'device.browserVersion': missing data
1. **'device.deviceCategory': high variance**
1. 'device.flashVersion': missing data  
1. **'device.isMobile': high variance (boolean)**
1. 'device.language': missing data 
1. 'device.mobileDeviceBranding': missing data 
1. 'device.mobileDeviceInfo': missing data 
1. 'device.mobileDeviceMarketingName': missing data  
1. 'device.mobileDeviceModel': missing data 
1. 'device.mobileInputSelector': missing data  
1. **'device.operatingSystem': high variance**
1. 'device.operatingSystemVersion': missing data  
1. 'device.screenColors': missing data
1. 'device.screenResolution': missing data  
1. 'geoNetwork.city: high variance (missing data)
1. 'geoNetwork.cityId': missing data
1. **'geoNetwork.continent': high variance**
1. **'geoNetwork.country': high variance**
1. 'geoNetwork.latitude': missing data 
1. 'geoNetwork.longitude': missing data  
1. 'geoNetwork.metro': high variance (missing data)
1. 'geoNetwork.networkDomain': high variance (missing data)
1. 'geoNetwork.networkLocation': missing data
1. 'geoNetwork.region': high variance (missing data)
1. **'geoNetwork.subContinent': high variance**
1. 'totals.bounces': high variance (boolean)
1. **'totals.hits': high variance**
1. 'totals.newVisits': high variance (boolean)
1. **'totals.pageviews': high variance**
1. **'totals.sessionQualityDim': high variance**
1. **'totals.timeOnSite': high variance**
1. **'totals.totalTransactionRevenue': high variance**
1. **'totals.transactionRevenue': high variance**
1. **'totals.transactions': high variance**
1. **'totals.visits': high variance**
1. 'trafficSource.adContent': high variance (missing data)
1. 'trafficSource.adwordsClickInfo.adNetworkType': high variance (missing data)
1. 'trafficSource.adwordsClickInfo.criteriaParameters': missing data
1. 'trafficSource.adwordsClickInfo.gclId': high variance
1. 'trafficSource.adwordsClickInfo.isVideoAd': low variance (missing data)
1. 'trafficSource.adwordsClickInfo.page': low variance (missing data)
1. 'trafficSource.adwordsClickInfo.slot':  low variance (missing data)
1. 'trafficSource.campaign': high variance (missing data)
1. 'trafficSource.isTrueDirect': low variance (missing data)
1. 'trafficSource.keyword': high variance (missing data)
1. 'trafficSource.medium': high variance (missing data)
1. 'trafficSource.referralPath': low variance (missing data)
1. 'trafficSource.source': high variance (missing data)

-- **Columns retained**

In [87]:
data_train = data_train.drop([
    'socialEngagementType', 
    'device.browserSize', 
    'device.browserVersion', 
    'device.flashVersion', 
    'device.language',
    'device.mobileDeviceInfo',
    'device.mobileDeviceBranding',
    'device.isMobile',
    'device.mobileDeviceInfo',                      
    'device.mobileDeviceMarketingName', 
    'device.mobileDeviceModel',
    'device.mobileInputSelector', 
    'device.operatingSystemVersion', 
    'device.screenColors',
    'device.screenResolution', 
    'geoNetwork.city',
    'geoNetwork.cityId',
    'geoNetwork.latitude',
    'geoNetwork.longitude', 
    'geoNetwork.metro',
    'geoNetwork.networkDomain',
    'geoNetwork.networkLocation',
    'geoNetwork.region',
    'trafficSource.adContent',
    'trafficSource.adwordsClickInfo.adNetworkType',
    'trafficSource.adwordsClickInfo.criteriaParameters',
    'trafficSource.adwordsClickInfo.gclId',
    'trafficSource.adwordsClickInfo.isVideoAd',
    'trafficSource.adwordsClickInfo.page',
    'trafficSource.adwordsClickInfo.slot',
    'trafficSource.campaign',
    'trafficSource.isTrueDirect',
    'trafficSource.keyword',
    'trafficSource.medium',
    'trafficSource.referralPath',
    'trafficSource.source'], axis=1) 

In [89]:
data_train_grouped= data_train.groupby(['date','channelGrouping', 'fullVisitorId', 'visitId', 'device.browser', 'device.deviceCategory',
        'device.operatingSystem','geoNetwork.continent', 'geoNetwork.country', 
       'geoNetwork.subContinent']).sum()

In [97]:
data_train_grouped;

In [91]:
data_train_grouped= data_train_grouped.reset_index()

* Creating training label from the 'totals.transactionRevenue' column

In [92]:
data_train_grouped['label']=data_train_grouped['totals.transactionRevenue']


In [93]:
data_train_grouped

,date,channelGrouping,fullVisitorId,visitId,device.browser,device.deviceCategory,device.operatingSystem,geoNetwork.continent,geoNetwork.country,geoNetwork.subContinent,...,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,totals.visits,label
0,2016-09-02,Affiliates,0063191589012692475,1472815091,Chrome,desktop,Windows,Europe,Croatia,Southern Europe,...,6.0,1.0,6.0,0.0,318.0,0.0,0.0,0.0,1.0,0.0
1,2016-09-02,Affiliates,0131179127719119947,1472813894,Firefox,desktop,Windows,Europe,Germany,Western Europe,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2016-09-02,Affiliates,0424951725842515190,1472822653,Chrome,desktop,Windows,Europe,United Kingdom,Northern Europe,...,6.0,1.0,6.0,0.0,250.0,0.0,0.0,0.0,1.0,0.0
3,2016-09-02,Affiliates,0693608817820417899,1472838961,Chrome,desktop,Windows,Americas,United States,Northern America,...,10.0,1.0,7.0,0.0,200.0,0.0,0.0,0.0,1.0,0.0
4,2016-09-02,Affiliates,0812798205180256552,1472853207,Chrome,desktop,Windows,Americas,United States,Northern America,...,5.0,1.0,5.0,0.0,23.0,0.0,0.0,0.0,1.0,0.0
5,2016-09-02,Affiliates,0857455342350014073,1472810732,Chrome,desktop,Windows,Europe,Spain,Southern Europe,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,2016-09-02,Affiliates,0918281628671165515,1472881096,Chrome,desktop,Windows,Africa,Nigeria,Western Africa,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,2016-09-02,Affiliates,1025336786416947475,1472814499,Chrome,desktop,Windows,Asia,India,Southern Asia,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,2016-09-02,Affiliates,1231966810799809141,1472806681,Chrome,desktop,Windows,Europe,United Kingdom,Northern Europe,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,2016-09-02,Affiliates,1241958385647224950,1472825974,Chrome,desktop,Windows,Americas,Brazil,South America,...,2.0,1.0,2.0,0.0,24.0,0.0,0.0,0.0,1.0,0.0


In [94]:
for i in data_train_grouped[data_train_grouped['label']>0].index :
    data_train_grouped.loc[i, 'label']= 1.0

In [95]:
data_train_grouped[data_train_grouped['label']>0] #conveters

,date,channelGrouping,fullVisitorId,visitId,device.browser,device.deviceCategory,device.operatingSystem,geoNetwork.continent,geoNetwork.country,geoNetwork.subContinent,...,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,totals.visits,label
131,2016-09-02,Direct,1776658355119092313,1472861525,Safari,desktop,Macintosh,Americas,United States,Northern America,...,19.0,1.0,16.0,0.0,538.0,4.027300e+08,395730000.0,1.0,1.0,1.0
274,2016-09-02,Direct,6194193421514403509,1472843572,Chrome,desktop,Linux,Americas,United States,Northern America,...,11.0,1.0,11.0,0.0,493.0,4.186000e+07,37860000.0,1.0,1.0,1.0
316,2016-09-02,Direct,747690048733385763,1472823219,Safari,desktop,Macintosh,Americas,United States,Northern America,...,33.0,0.0,24.0,0.0,1185.0,4.048000e+07,34480000.0,1.0,1.0,1.0
454,2016-09-02,Organic Search,0643132668463442280,1472867400,Chrome,desktop,Windows,Americas,United States,Northern America,...,67.0,1.0,44.0,0.0,982.0,3.268000e+07,25680000.0,1.0,1.0,1.0
732,2016-09-02,Organic Search,3794016165791476291,1472832496,Safari,mobile,iOS,Americas,United States,Northern America,...,28.0,1.0,20.0,0.0,387.0,1.203400e+08,117340000.0,1.0,1.0,1.0
894,2016-09-02,Organic Search,5327166854580374902,1472844906,Chrome,desktop,Macintosh,Americas,United States,Northern America,...,11.0,0.0,10.0,0.0,322.0,3.136700e+08,306670000.0,1.0,1.0,1.0
956,2016-09-02,Organic Search,6045930241388777741,1472825504,Chrome,desktop,Windows,Americas,United States,Northern America,...,22.0,0.0,16.0,0.0,530.0,9.050000e+07,81500000.0,1.0,1.0,1.0
1003,2016-09-02,Organic Search,6712520540378436905,1472853233,Chrome,desktop,Chrome OS,Americas,United States,Northern America,...,38.0,1.0,31.0,0.0,913.0,3.967000e+07,33670000.0,1.0,1.0,1.0
1037,2016-09-02,Organic Search,7147112211830167925,1472853332,Chrome,mobile,Android,Americas,United States,Northern America,...,22.0,1.0,16.0,0.0,429.0,3.648000e+07,35480000.0,1.0,1.0,1.0
1107,2016-09-02,Organic Search,8007300615127214527,1472850417,Chrome,desktop,Macintosh,Americas,United States,Northern America,...,34.0,0.0,30.0,0.0,688.0,4.139000e+07,35390000.0,1.0,1.0,1.0


* We notice that only a small fraction of the visitors convert. This leads to a class imbalance that needs to be addressed before training our propensity to convert model.

In [96]:
data_train_grouped[data_train_grouped['label']<=0] #non-converters

,date,channelGrouping,fullVisitorId,visitId,device.browser,device.deviceCategory,device.operatingSystem,geoNetwork.continent,geoNetwork.country,geoNetwork.subContinent,...,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,totals.visits,label
0,2016-09-02,Affiliates,0063191589012692475,1472815091,Chrome,desktop,Windows,Europe,Croatia,Southern Europe,...,6.0,1.0,6.0,0.0,318.0,0.0,0.0,0.0,1.0,0.0
1,2016-09-02,Affiliates,0131179127719119947,1472813894,Firefox,desktop,Windows,Europe,Germany,Western Europe,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2016-09-02,Affiliates,0424951725842515190,1472822653,Chrome,desktop,Windows,Europe,United Kingdom,Northern Europe,...,6.0,1.0,6.0,0.0,250.0,0.0,0.0,0.0,1.0,0.0
3,2016-09-02,Affiliates,0693608817820417899,1472838961,Chrome,desktop,Windows,Americas,United States,Northern America,...,10.0,1.0,7.0,0.0,200.0,0.0,0.0,0.0,1.0,0.0
4,2016-09-02,Affiliates,0812798205180256552,1472853207,Chrome,desktop,Windows,Americas,United States,Northern America,...,5.0,1.0,5.0,0.0,23.0,0.0,0.0,0.0,1.0,0.0
5,2016-09-02,Affiliates,0857455342350014073,1472810732,Chrome,desktop,Windows,Europe,Spain,Southern Europe,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,2016-09-02,Affiliates,0918281628671165515,1472881096,Chrome,desktop,Windows,Africa,Nigeria,Western Africa,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,2016-09-02,Affiliates,1025336786416947475,1472814499,Chrome,desktop,Windows,Asia,India,Southern Asia,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,2016-09-02,Affiliates,1231966810799809141,1472806681,Chrome,desktop,Windows,Europe,United Kingdom,Northern Europe,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,2016-09-02,Affiliates,1241958385647224950,1472825974,Chrome,desktop,Windows,Americas,Brazil,South America,...,2.0,1.0,2.0,0.0,24.0,0.0,0.0,0.0,1.0,0.0
